In [1]:
import pandas as pd
import json
from datetime import datetime
import os
import re

In [2]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 


# 1- cleaning data

In [3]:
df = pd.read_csv('khakha.csv')
dfwin = pd.read_csv('../AwardDates.csv')
data = dict()
with open('../dates.json') as json_file:
    data = json.load(json_file)
for d in data: 
    date = d['date']
    result = re.search('Published(.*)', date)
    partial = result.group(1)
    if('by' in partial) :
        partial = re.search('(.*)by', partial)
        d['date'] = partial.group(1).lstrip().rstrip()
    else:
        d['date'] = partial.lstrip().rstrip()

with open('new.json', 'w') as new:
    json.dump(data, new)

In [4]:
# calculating dates and matching isbn
data = dict()
with open('new.json') as json_file:
    data = json.load(json_file)
hi = my_dictionary()
h2 = my_dictionary()
isbns = list()
for d in data:

    ds = str(d['date']).split()
    if len(ds)>1:
        ds[1]=ds[1].replace('th','').replace('st','').replace('rd','').replace('nd','')
        d['date'] = ''.join(str(e)+' ' for e in ds)
    if len(ds)>3:
         d['date'] = ''.join([ds[0],' ',ds[1],' ',ds[2],' '])
    try:
        d['date']=datetime.strptime(d['date'], '%B %d %Y ')
    except ValueError :
        try:
           d['date']= datetime.strptime(d['date'], '%B %Y ')
        except ValueError :
           d['date']= datetime.strptime(d['date'], '%Y')
    if(df[df['url']==d['url']].empty):
        continue
    year=df[df['url']==d['url']]['year'].iloc[0]
    win=datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')
    if((win -  d['date']).days<1):
        d['date']=(win -  d['date']).days
    else:
        d['date']=(win -  d['date']).days 
#     if (d['name'] == 'Calling Dr. Laura'):
#         print(d['date'])
#         print(win)
    hi.add(d['url'], d['date'])
    h2.add(d['url'], d['isbn'])
    isbns.append(d['isbn'])

# merging data and adding 'times_appeared_NYTime','avgrank_NYTime','bestRank','listnames' to are coulems

In [5]:
dfnew = pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
df.merge(dfnew, on='url', how='left').merge(dfnew1, on='url', how='left').to_csv("withdays.csv")
df = pd.read_csv('withdays.csv')
df = df.reindex(df.columns.tolist() + ['times_appeared_NYTime','avgrank_NYTime','bestRank','listnames'], axis=1)

In [6]:
def set_default_values():
    values = {}
    values['bestrank'] = float('inf')
    values['sum_ranks'] =  float('inf')
    values['count'] = 0
    values['listnames'] = set()
    return values

def set_row(myDf, values, isbn, year, key):
    myDf.loc[(myDf[key] == isbn) & (myDf['year'] == year), 'times_appeared_NYTime'] = values['count']
    if((values['count'] == 0) or (values['sum_ranks'] == float('inf'))):
        avg = float('inf')
    else:
        avg =  values['sum_ranks'] / values['count']
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'avgrank_NYTime'] = avg
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'bestRank'] = values['bestrank']
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'listnames'] = len(values['listnames'])

reading newYork Data and adding to feilds

In [7]:
def add_nyt_data(data, df, isbns, key):
    errors = 0
    for i in isbns:
        print(i)
        # will be handled later
        if i =='English': continue         
        # for each year, we need to calculate coloumn values 
        years = df[df[key] == i]['year']
        for year in years:
            values = {'bestrank':float('inf'), 'sum_ranks':0, 'count':0, 'listnames' : set()}
            # didn't appear in nyt list   
            if len(data[i]['results'])==0:    
                values = set_default_values()
                set_row(df, values, i, year, key)
                continue

            win = datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')
            
            for z in data[i]['results'][0]['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d') < win):
                    values['sum_ranks'] += z['rank']
                    values['count'] = values['count'] + 1
                    values['listnames'].add(z['list_name'])
                    if values['bestrank'] > z['rank']:
                        values['bestrank'] = z['rank']
                        
            if values['count'] > 0:
                set_row(df, values, i, year, key)
            else:
                values = set_default_values()
                set_row(df, values, i, year, key)

    print(errors)

In [8]:
data = dict()
newYork_dir = "../newYork Data/"
data = dict()
new_data = dict()
for filename in os.listdir(newYork_dir):
    if (filename.endswith(".json") and filename.startswith("res")): 
        cmplt_path = os.path.join(newYork_dir, filename)
        print(cmplt_path)
        with open(cmplt_path) as json_file :
            new_data = json.load(json_file)
            data.update(new_data)
    else:
        continue

print(len(isbns))
key = 'isbn'
add_nyt_data(data, df, isbns, key)

../newYork Data/res8.json
../newYork Data/res4.json
../newYork Data/res5.json
../newYork Data/res10.json
../newYork Data/res9.json
../newYork Data/res2.json
../newYork Data/res3.json
../newYork Data/res1.json
../newYork Data/res6.json
../newYork Data/res7.json
3969
159463176X
0670026638
English
0670026603
1439102767
1476729085
0812993802
0399159347
0061941026
0307265749
0345532740
0670026611
1451661509
1594488398
1400067685
1594204780
0988839407
0473235501
0007489943
1455519146
English
038553521X
080715086X
1555976352
1595588760
0957466129
193822101X
0374173613
1935738496
English
0763660531
0062017810
English
0316187488
1250012570
1250012570
1423159527
0062234897
1452110069
0545436796
0062203096
0763646016
0811867838
0811872351
1627530673
1596437316
0547773579
0316182710
0307931900
0670785601
English
006223336X
1423166000
B00BMKH5NW
0373210639
1250021529
1250021529
0062235656
0316125873
1423183096
0062217135
161963144X
0316221333
English
1423146727
0316200964
1607066874
0547615590
1401

067002502X
0061561630
0385535783
0451463811
0802120202
1937475018
1841498890
0756407117
0345523474
1616960922
0316187216
0812992970
0061928127
1616200421
0307959945
0060885599
0312641893
0312641893
006207203X
0545424933
0763655996
0375867562
1596433973
English
0062059939
1423113349
141970379X
1250001544
1250001544
0425256138
B0085ZHMA8
0345803507
0345803507
0425265951
0425265951
0316176192
1423161610
0763654078
0545298415
0062117432
1780334834
0062003216
0316204277
0983613117
0307888878
B0080K3N42
031618201X
193351759X
1612182798
0984475257
0307957314
0375424334
1401234658
0374386153
0393081192
1423147944
0316122386
0761169253
1599908441
1478275219
145162445X
0307377903
B0096SYACG
1401310605
0307720454
159420411X
160358286X
1936608758
0062103296
1579654584
0307990834
1400067669
0307888754
1439127700
0670023051
0812992784
0525953728
0307719081
0399159460
0062112031
1470177226
1451642997
1597802301
1841499005
0857662309
034551128X
0307595951
English
0441020712
0316212377
9781101585191
03

1419729071
0385539304
0393635066
0525536515
English
0812998839
0374280002
0385542844
194883653X
198481706X
English
0735210934
0393652246
1524743011
1250301696
1250301696
English
0385544235
0385544235
0399590595
0399590595
0525619240
0525562028
0525562028
0525656154
1501196014
1501196014
0385544251
0385544251
English
English
0062898108
125020058X
125020058X
0374311544
0374311544
1250313198
1250313198
1538715600
1538715600
0062906372
073522370X
0306845989
1250147603
1524732710
0062049534
1501132733
0525657681
1250019958
0525561587
1250237238
1635571855
0393651649
0525511350
1328631346
1496441338
0062876880
198481799X
1524732621
1946885983
0399581995
1683691555
0316420573
1328482472
132400262X
1635653703
1328557472
1408867761
0451497015
0062821385
1400209609
1328662055
0062888439
1250178959
0316478520
1451642296
0525560343
1250313570
0316449881
1982113227
1555978274
0525575472
0525510540
1620974363
0525509283
1501134612
1328663817
073522529X
English
150118220X
1250130336
1594633150
150116

1594204799
1938912489
0062289578
0143126520
1771680040
0374191875
1933517891
B00JHIUOAW
1555976905
1566893747
0316401080
0809333252
English
1607069318
1616552956
1612629415
1936976552
1927925185
1607069466
1401246087
0345529375
1442465956
1607069385
1607069458
141971189X
1596436581
1596436972
1484724917
0316406813
English
0399252517
0545552702
1442487674
0062104926
1419710206
0385744250
0545521459
141971144X
142316492X
English
0399164057
1594746125
1481426303
0312642970
1619630656
0062285688
B00DB2YN0C
1595143211
English
0345539788
0345539788
0345539788
0399167838
1423171020
0374384681
0763665665
1607742675
English
1623363586
1449450334
English
1583335277
067001544X
1451655096
145165619X
0804177872
B00I6W7O5S
1628600195
1594204071
0062267337
0385345623
1607746212
039916927X
0062218360
076117897X
0812993012
0393244660
067443000X
1591845327
0374228442
1591847788
0804140847
B00H26IFJS
0062273205
1620408120
1591846358
0804137382
1250020204
1613741049
1476725764
0062092898
0307956768
038553

0756410606
0553419730
006228925X
1635762111
1545117438
0062389025
0062654195
1476716730
1455563935
1978098758
0385541457
0062563661
1477848665
0062430246
1542722829
0544968840
125011117X
1250069793
081299860X
1250074134
0062491792
0345544951
1400067693
1400068320
1594634637
0062359983
110194661X
110194661X
B01IC9BQT4
1250075610
0307270645
1101883073
1101883073
English
1250078067
1250078067
0451477537
0451477537
B01IDN23RI
1250085470
1501126229
1501126229
0399184511
0062380753
1786810026
0399176772
1492626015
0812998480
1631520342
1631491768
1101981202
1250078067
1250078067
1449480128
1532913680
English
1455565288
0544570987
144947411X
0692715711
1449480357
1449481019
031638660X
1551526417
B01B1YPOZY
0913785768
1555977448
0062377019
148473274X
0399187197
1423160924
1101994762
0553510401
0763681172
0545522315
1101994827
1484709675
0545685443
0316381993
0553536745
0062333461
0765381400
1619634465
1619634465
1442468351
English
0062392174
0545424984
0062310666
1250044650
9780008250447
14814

0312583729
0983360766
B004KSQVCO
0061876895
0307408841
1451648537
0679456724
0316132942
0061732370
1416571760
0670022209
0307592200
1400040159
0385526261
0307463907
1594487871
0805093079
0307265722
1594487979
0399157441
0451233166
0061351520
1612130291
0373775814
1439136807
0425242099
1615819495
0062021273
044657449X
0425240517
English
English
0425241505
158479903X
0446557315
0525951997
140006872X
1606419315
0738214930
0307272702
1579654363
1449401090
0307465322
1592406017
1602861331
1580082777
1416583939
0738212741
1594856370
1401302025
0061711527
159420229X
1614520011
0525952330
1593156596
140006760X
0393081818
0385533063
1439192812
1936719002
0307588688
1416596585
0618883029
1594203008
1423140567
1423140591
1423140591
0061998168
0545027896
0061555185
1406311529
0312649614
0399250573
141699792X
0375858512
1607461102
0547152604
0375868704
0062015052
1442407689
1439102724
1439102724
1442403543
1442403543
1442403543
English
English
0385534639
0385534639
030788743X
030788743X
030788743X


saving data

In [9]:
df.to_csv("partial.csv")

cheak nall values

In [10]:
df = pd.read_csv('partial.csv')
def nans(df): return df[df.isnull().any(axis=1)]
nans(df)

,Unnamed: 0,Unnamed: 0.1,average rating,average_rating_w,category,name,num 1 stars,num 2 stars,num 3 stars,num 4 stars,...,url,winner,writer,year,daystocom,isbn,times_appeared_NYTime,avgrank_NYTime,bestRank,listnames
0,0,0,3.847458,3.99,Graphic Novels & Comics,"The Umbrella Academy, Vol. 3: Hotel Oblivion",0.013559,0.077966,0.294915,0.274576,...,https://www.goodreads.com/book/show/21801668-t...,0,"['Gerard Way', 'Gabriel Bá']",2019,84.0,English,NaN,NaN,NaN,NaN
3,3,3,3.371930,4.20,Graphic Novels & Comics,Shades of Magic Vol. 1: The Steel Prince,0.157895,0.108772,0.242105,0.185965,...,https://www.goodreads.com/book/show/38524301-s...,0,"['V.E. Schwab', 'Andrea Olimpieri']",2019,279.0,English,NaN,NaN,NaN,NaN
4,4,4,3.808874,3.82,Graphic Novels & Comics,Teen Titans: Raven,0.023891,0.044369,0.279863,0.402730,...,https://www.goodreads.com/book/show/38452813-t...,0,"['Kami Garcia', 'Gabriel Picolo']",2019,161.0,English,NaN,NaN,NaN,NaN
7,7,7,4.367347,3.99,Graphic Novels & Comics,"""Okay""",0.000000,0.010204,0.115646,0.370748,...,https://www.goodreads.com/book/show/42835876-o...,0,"['Kieron Gillen', 'Jamie McKelvie', 'Matt Wils...",2019,63.0,English,NaN,NaN,NaN,NaN
10,10,10,4.305085,4.20,Graphic Novels & Comics,"Fence, Vol. 3",0.003390,0.013559,0.111864,0.416949,...,https://www.goodreads.com/book/show/40538764-f...,0,"['C.S. Pacat', 'Johanna the Mad', 'Joana LaFue...",2019,112.0,English,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,3537,3537,4.372483,4.18,Fantasy,The Obelisk Gate,0.020134,0.006711,0.067114,0.392617,...,https://www.goodreads.com/book/show/26228034-t...,0,['N.K. Jemisin'],2016,110.0,English,NaN,NaN,NaN,NaN
3539,3539,3539,4.576667,4.28,Fantasy,The Curse of Tenth Grave,0.000000,0.003333,0.050000,0.313333,...,https://www.goodreads.com/book/show/26114340-t...,0,['Darynda Jones'],2016,161.0,English,NaN,NaN,NaN,NaN
3586,3586,3586,4.265993,4.19,Historical Fiction,America's First Daughter,0.010101,0.013468,0.134680,0.383838,...,https://www.goodreads.com/book/show/25817162-a...,0,"['Stephanie Dray', 'Laura Kamoie']",2016,280.0,English,NaN,NaN,NaN,NaN
3611,3611,3611,4.070234,3.90,Science & Technology,The Inevitable: Understanding the 12 Technolog...,0.026756,0.030100,0.173913,0.384615,...,https://www.goodreads.com/book/show/27209431-t...,0,['Kevin Kelly'],2016,182.0,English,NaN,NaN,NaN,NaN


read data from json 11 with different struct and saving to csv

In [11]:
key = 'url'
with open('../newYork Data/remained.json') as json_file:
    data = json.load(json_file)
    for i in data:        
        years = df[df[key] == i]['year']
        for year in years:
            # didn't appear in any list
            if len(data[i]['results']) == 0:
                values = set_default_values()
                set_row(df, values, i, year, key)
                continue

            values = {'bestrank':float('inf'), 'sum_ranks':0, 'count':0, 'listnames' : set()}
            win=datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')

            for z in data[i]['results'][0]['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    values['sum_ranks'] += z['rank']
                    values['count'] = values['count'] + 1
                    values['listnames'].add(z['list_name'])
                    if values['bestrank'] > z['rank']:
                        values['bestrank'] = z['rank']
            if values['count'] > 0:
                set_row(df, values, i, year, key)
            else:
                values = set_default_values()
                set_row(df, values, i, year, key)
df.to_csv("final.csv")

saving nulls in file

In [12]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(df).to_csv("nulls.csv")

In [13]:
# dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
# dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
# # dfnew2 = pd.DataFrame(h5.items(), columns=['isbn', 'times_appred_NYTime'])
# # dfnew3 = pd.DataFrame(h4.items(), columns=['isbn', 'avgrank_NYTime'])
# # dfnew4 = pd.DataFrame(h6.items(), columns=['isbn', 'bestRank'])
# # dfnew5 = pd.DataFrame(h7.items(), columns=['isbn', 'listnames'])
# df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
# # .merge(dfnew3,on='isbn', how='left').merge(dfnew4,on='isbn', how='left').merge(dfnew5,on='isbn', how='left').to_csv("withdays.csv")

In [ ]:
df = pd.read_csv('../goodreads_output3.csv')
df

In [ ]:
df1 = pd.read_csv('../test.csv')
new = df1[df1['year']== 2019].drop(['daysToContest', 'isbn', 'times_appred_NYTime', 'avgrank_NYTime', 'bestRank', 'listnames'], axis = 1)
def add(a, b, c, d, e): 
    return a + b + c
new[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']] = new[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']].div(new.iloc[:, -10:-5].sum(axis=1), axis=0)
new

In [ ]:
before = pd.read_csv('../goodreads_output3.csv')
df.columns
output =  new.append(before) 
ourput = output.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1)

In [ ]:
 ourput

In [ ]:
ourput.to_csv('khakha.csv', index=False)

In [ ]:
#df = pd.read_csv('final.csv')
#df[df['avgrank_NYTime'] < 100]

In [ ]:
#df

In [ ]:
df[['name', 'category']]